--Install libraries--

In [ ]:
!pip install spacy
!pip install newspaper3k
!pip install wordcloud
!pip install textblob
!pip install seaborn
!python -m spacy download en_core_web_sm

--Load dependencies--

In [ ]:
import spacy
import pandas as pd
import json
import nltk
import en_core_web_sm
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize.regexp import WhitespaceTokenizer
import newspaper
from newspaper import article
import matplotlib.pyplot as plt
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from textblob import TextBlob
import base64
import string
import re
from collections import Counter
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from spacy.lang.en import English
parser = English()
import numpy as np

--Read in CSV File--

In [ ]:
df = pd.read_csv (r'test_agor.csv')
new_df = df[['Title', 'Body']] 
new_df.head()

--Remove punctuation and stopwords--

In [ ]:
punctuations = string.punctuation
useless_words = nltk.corpus.stopwords.words("english") + list(punctuations)

--Remove additional "noisy" text--

In [ ]:
useless_words_extended = useless_words.copy()
useless_words_extended.extend(["n't","'m","'s", "'ve", "'ll", "'re", "...", "..", '``', "''"])

--Create the raw corpus--

In [ ]:
a = new_df['Title'].to_list()
b = new_df['Body'].to_list()
corpus = [''.join(map(str, i)) for i in zip(a, b)]

# loop through each element in the list (in this case the list made of string elements) and tokenize each line and add it to the list of words
list_of_words = []
for line in corpus:
    list_of_words.extend(nltk.word_tokenize(line.lower()))
list_of_words

--Apply code to clean corpus--

In [ ]:
filtered_words = [word for word in list_of_words if not word in useless_words_extended]
filtered_words

--obtain word counts--

In [ ]:
word_counter = Counter(filtered_words)
word_counter

--lemmatize data--

In [ ]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

lemmatized_wordslist = []
for word in filtered_words:
    lem_str = lemmatizer.lemmatize(word)
    lemmatized_wordslist.append(lem_str)

lemmatized_wordslist

--pass cleaned and lemmatized data through spaCy nlp pipeline--

In [ ]:
nlp = en_core_web_sm.load()

cleaned_wordlist = nlp(str(lemmatized_wordslist))


--count adjectives in the data--

In [ ]:
token_list = [[token,token.pos_] for token in cleaned_wordlist]
tk = pd.DataFrame(token_list,columns = ['word','POS'])
tk = tk[tk.POS != 'PUNCT']
tk_adj = tk[tk.POS == 'ADJ']

l1 = [g1.text for g1 in tk_adj.word.tolist()]
d =Counter(l1)
agora_adj_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
agora_adj_df.columns = ['word','count']

agora_adj_df['POS'] =pd.Series(['ADJ' for i in range(agora_adj_df.shape[0])])
agora_adj_df = agora_adj_df.sort_values(by = 'count', ascending = False)
agora_adj_df.iloc[0:45]

--count nouns  in the data--

In [ ]:
tk = tk[tk.POS != 'PUNCT']
tk_noun = tk[tk.POS == 'NOUN']
l1 = [g1.text for g1 in tk_noun.word.tolist()]
d =Counter(l1)
agora_noun_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
agora_noun_df.columns = ['word','count']

agora_noun_df['POS'] =pd.Series(['NOUN' for i in range(agora_noun_df.shape[0])])
agora_noun_df = agora_noun_df.sort_values(by = 'count', ascending = False)
agora_noun_df.iloc[0:45]

--count verbs in the data--

In [ ]:
l1 = [g1.text for g1 in tk_verb.word.tolist()]
d =Counter(l1)
agora_verb_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
agora_verb_df.columns = ['word','count']
agora_verb_df.sort_values(by='count', ascending=False)
agora_verb_df['POS'] =pd.Series(['VERB' for i in range(agora_verb_df.shape[0])])
agora_verb_df = agora_verb_df.sort_values(by = 'count', ascending = False)
agora_verb_df.iloc[:45]



--get and count trigrams in the data--

In [ ]:
blob = TextBlob(clean_doc)
blob3 = blob.ngrams(n=3)
flat_blob3 = [bb[0]+' '+bb[1]+' ' +bb[2] for bb in blob3]
Counter(flat_blob3).most_common(100)

--calculate tfidf for unigrams--

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(corpus)
# print(word_count_vector)

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)


# print idf values
df_idf = pd.DataFrame(zip(cv.get_feature_names(),tfidf_transformer.idf_), columns=["word","idf_weights"])
 
# sort ascending
df_idf.sort_values(by='idf_weights',ascending=False)

df_idf.to_csv('test_idf.csv')
df_idf.sort_values(by='idf_weights',ascending=True)

In [ ]:
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
 
# just send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(corpus)

In [ ]:
first_document_vector = tfidf_vectorizer_vectors[0]
df = pd.DataFrame(first_document_vector.T.todense(),  columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)